# 공급량 데이터를 누적하여 저장하는 프로그램
+ 작성: 김지훈
+ 수정: 임경호

## sql 데이터 추출

In [1]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

# 공급량 누적 테이블
query = 'SELECT date_id FROM supply_cumsum ORDER BY date_id DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
sp_sum_last_date = row[0]     # 공급량누적 테이블에 저장된 데이터의 마지막 일자

# 공급량 테이블
query = 'SELECT date_id FROM supply WHERE date_id > %s ORDER BY date_id'
cur.execute(query, sp_sum_last_date)
rows = cur.fetchall()

beg_date = rows[0][0]
end_date = rows[-1][0]

conn.close()

In [2]:
print(f'누적 테이블에는 {sp_sum_last_date}까지 데이터가 저장되어 있으며, 이 날짜 이후에 공급량 테이블에는 {beg_date}부터 {end_date}까지 데이터가 있음')

누적 테이블에는 20230630까지 데이터가 저장되어 있으며, 이 날짜 이후에 공급량 테이블에는 20230701부터 20230831까지 데이터가 있음


아래 내용은 db내에 적재 되어있는 <b>date</b>와 <b>supply</b> 테이블을 활용해서 <b>SQL 쿼리문</b>으로 작성

In [3]:
import pandas as pd

engine = dbconnect.db_engine("DEMO_DM")
query = """
SELECT s.date_id AS date_id
		, supply_mj AS day_mj
		, SUM(sum(supply_mj)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y-%%m') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS month_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY quarter_6,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_6_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY quarter_4,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_4_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS  year_mj
		, supply_m3 AS day_m3
		, SUM(sum(supply_m3)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y-%%m') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS month_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY quarter_6,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_6_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY quarter_4,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_4_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS  year_m3
		FROM mdw.date AS d
			INNER JOIN mdw.supply AS s on s.date_id = d.date_id and s.date_id between %(date1)s and %(date2)s
	group BY DATE_FORMAT(date_type, '%%Y-%%m-%%d') ;
"""

# sql execution with parameters
df_data = pd.read_sql(sql=query, con=engine, params={'date1' : beg_date, 'date2' : end_date})
df_data #테이블 확인

,date_id,day_mj,month_mj,quarter_6_mj,quarter_4_mj,year_mj,day_m3,month_m3,quarter_6_m3,quarter_4_m3,year_m3
0,20230701,54334272,5.433427e+07,5.433427e+07,5.433427e+07,5.433427e+07,1271584,1271584.0,1271584.0,1271584.0,1271584.0
1,20230702,47366511,1.017008e+08,1.017008e+08,1.017008e+08,1.017008e+08,1110437,2382021.0,2382021.0,2382021.0,2382021.0
2,20230703,68812651,1.705134e+08,1.705134e+08,1.705134e+08,1.705134e+08,1616317,3998338.0,3998338.0,3998338.0,3998338.0
3,20230704,72875696,2.433891e+08,2.433891e+08,2.433891e+08,2.433891e+08,1709005,5707343.0,5707343.0,5707343.0,5707343.0
4,20230705,71794995,3.151841e+08,3.151841e+08,3.151841e+08,3.151841e+08,1682655,7389998.0,7389998.0,7389998.0,7389998.0
...,...,...,...,...,...,...,...,...,...,...,...
57,20230827,46032880,1.599507e+09,3.553166e+09,3.553166e+09,3.553166e+09,1085493,37595878.0,83375362.0,83375362.0,83375362.0
58,20230828,67590261,1.667097e+09,3.620756e+09,3.620756e+09,3.620756e+09,1593011,39188889.0,84968373.0,84968373.0,84968373.0
59,20230829,71823021,1.738920e+09,3.692579e+09,3.692579e+09,3.692579e+09,1690532,40879421.0,86658905.0,86658905.0,86658905.0
60,20230830,70581615,1.809502e+09,3.763161e+09,3.763161e+09,3.763161e+09,1660557,42539978.0,88319462.0,88319462.0,88319462.0


In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_id       62 non-null     object 
 1   day_mj        62 non-null     int64  
 2   month_mj      62 non-null     float64
 3   quarter_6_mj  62 non-null     float64
 4   quarter_4_mj  62 non-null     float64
 5   year_mj       62 non-null     float64
 6   day_m3        62 non-null     int64  
 7   month_m3      62 non-null     float64
 8   quarter_6_m3  62 non-null     float64
 9   quarter_4_m3  62 non-null     float64
 10  year_m3       62 non-null     float64
dtypes: float64(8), int64(2), object(1)
memory usage: 5.5+ KB


In [5]:
df_data.columns

Index(['date_id', 'day_mj', 'month_mj', 'quarter_6_mj', 'quarter_4_mj',
       'year_mj', 'day_m3', 'month_m3', 'quarter_6_m3', 'quarter_4_m3',
       'year_m3'],
      dtype='object')

## DB 내 테이블 적재

In [6]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_data.itertuples():
    sql = "insert into supply_cumsum(date_id, 일_mj, 월_mj, 6분기_mj, 4분기_mj, 연_mj, \
                                              일_m3, 월_m3, 6분기_m3, 4분기_m3, 연_m3) \
                              values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11]))

conn.commit()
conn.close()